In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df = pd.read_csv('../../data/merged_data_sentiment.csv')
df.head()

/var/folders/4c/w2fyysz50_j_234jdvb_c9wr0000gn/T/ipykernel_37713/4278760478.py:5: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../../data/merged_data_sentiment.csv')


,ID,Content,User,Date,Location,Reactions,N_Children,Post Title,Platform,meta,lang,Subreddit,Unnamed: 0,Sentiment
0,UgyDawiDJRU5j8e9g6x4AaABAg,i love my mum~!TAT,carol lui,2013-04-30,NaN,1.0,0.0,AI - ママへ (short ver.) Music Video,youtube,"{'vid_id': 'JjVqw-rKqJo', 'user_id': 'UCgxe0Tv...",en,NaN,NaN,"0.9998131394386292,0.9847044944763184,0.992429..."
1,Ugx5UKqZcGpX0Qi8art4AaABAg,I was in tears until the lady gave her mother ...,C Ladner,2013-05-13,NaN,0.0,0.0,AI - ママへ (short ver.) Music Video,youtube,"{'vid_id': 'JjVqw-rKqJo', 'user_id': 'UC98Fh4X...",en,NaN,NaN,"-0.8607058525085449,-0.4791601002216339,0,0.0,..."
2,UgybIB4C2ppmGqU40WV4AaABAg,"I like her voice!! So deep, i thought the voic...",joseph jr.,2013-05-24,PH,0.0,0.0,AI - ママへ (short ver.) Music Video,youtube,"{'vid_id': 'JjVqw-rKqJo', 'user_id': 'UCUe_b0X...",en,NaN,NaN,"0.9991986155509949,0.9696046113967896,0.984618..."
3,Ugyi1lQIQnmZ2jVwk4B4AaABAg,A verve and flow equal to Maria Carrey's debut...,medusanerve,2013-05-28,NaN,1.0,0.0,AI - ママへ (short ver.) Music Video,youtube,"{'vid_id': 'JjVqw-rKqJo', 'user_id': 'UCILrk7Q...",en,NaN,NaN,"0.9990953207015991,0.901877224445343,0.9919101..."
4,Ugw2iIVWPilqFB7RMEl4AaABAg,great voice:D\n,ramon goedendorp,2013-06-08,NaN,1.0,0.0,AI - ママへ (short ver.) Music Video,youtube,"{'vid_id': 'JjVqw-rKqJo', 'user_id': 'UCNs25ls...",en,NaN,NaN,"0.9998722076416016,0.9386110305786133,0.989294..."


In [2]:
# only 2019 and onwards (sorted based on the Date column)
# df = df[df['Date'] >= '2019-01-01'].sort_values(by='Date')
# df.shape

In [3]:
# get mean sentiment for each comment
def avg_sent(df):
  for i, row in df.iterrows():
    sent = row['Sentiment'].split(',')
    sent = [float(s) for s in sent]
    df.at[i, 'Avg_Sentiment'] = sum(sent) / len(sent)
avg_sent(df)

In [4]:
# show line chart of sentiment over time (by month)

# get month from date
df['Month'] = pd.to_datetime(df['Date']).dt.to_period('M').astype(str)

# compute volume (number of rows in the df)
df['Volume'] = 1

# group by month and get mean sentiment and also sum of volume
df = df.groupby('Month').agg({'Avg_Sentiment': 'mean', 'Volume': 'sum'}).reset_index()

# plot both the average sentiment and the volume (same chart, two y axes to allow for different scales)
# (secondary y-axis with subplots, use add_trace() to add the second trace)
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    
    go.Scatter(x=df['Month'], y=df['Avg_Sentiment'], name="Average Sentiment"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df['Month'], y=df['Volume'], name="Volume"),
    secondary_y=True,
)

fig.update_layout(
    title_text="Average Sentiment and Volume by Month",
    xaxis_title="Month",
    yaxis_title="Average Sentiment",
    yaxis2_title="Volume",
    legend_title="Legend Title",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)

fig.show()

/var/folders/4c/w2fyysz50_j_234jdvb_c9wr0000gn/T/ipykernel_37713/3509122312.py:4: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['Month'] = pd.to_datetime(df['Date']).dt.to_period('M').astype(str)


In [5]:
# what are the most popular posts during each spike?
spike_month = '2017-08'
events_df = pd.read_csv('../../data/merged_data_sentiment.csv')
events_df['Month'] = pd.to_datetime(events_df['Date']).dt.to_period('M').astype(str)
events_df = events_df[events_df['Month'] == spike_month]
events_df = events_df.groupby('Post Title').agg({'ID': 'count'}).reset_index()
events_df = events_df.sort_values(by='ID', ascending=False)
events_df.head()

/var/folders/4c/w2fyysz50_j_234jdvb_c9wr0000gn/T/ipykernel_37713/1367360031.py:3: DtypeWarning:

Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.

/var/folders/4c/w2fyysz50_j_234jdvb_c9wr0000gn/T/ipykernel_37713/1367360031.py:4: UserWarning:

Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.



,Post Title,ID
2,"Artificial Intelligence Went Rogue, Created It...",45
6,Geordie Rose | Quantum Computing: Artificial I...,9
0,A.I.SHA My Virtual Girlfriend Season 2 | Episo...,7
4,Drones Getting Smarter with AI | Intel,6
1,AI - Story,4


In [6]:
baseline_df = pd.read_csv('../../data/merged_data_sentiment.csv')

# get month from date
baseline_df['Month'] = pd.to_datetime(baseline_df['Date']).dt.to_period('M').astype(str)

# compute volume (number of rows in the df)
baseline_df['Volume'] = 1

# group by month and get mean sentiment and also sum of volume
baseline_df = baseline_df.groupby('Month').agg({'Volume': 'sum'}).reset_index()

# exclude data from July 2019, October 2020, August 2021, and the year 2023
baseline_df = baseline_df[~baseline_df['Month'].isin(['2019-07', '2020-10', '2021-08'])]
baseline_df = baseline_df[~baseline_df['Month'].str.contains('2023')]

# get median volume in each year
baseline_df['Year'] = pd.to_datetime(baseline_df['Month']).dt.to_period('Y').astype(str)
baseline_df = baseline_df.groupby('Year').agg({'Volume': 'median'}).reset_index()
baseline_df.T

/var/folders/4c/w2fyysz50_j_234jdvb_c9wr0000gn/T/ipykernel_37713/604474362.py:1: DtypeWarning:

Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.

/var/folders/4c/w2fyysz50_j_234jdvb_c9wr0000gn/T/ipykernel_37713/604474362.py:4: UserWarning:

Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.



,0,1,2,3,4,5,6,7,8,9
Year,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Volume,1.0,5.5,16.0,29.5,41.5,104.0,230.0,754.0,298.0,339.5


In [8]:
# get average sentiment in each year
avg_yr_df = pd.read_csv('../../data/merged_data_sentiment.csv')
avg_yr_df['Year'] = pd.to_datetime(avg_yr_df['Date']).dt.to_period('Y').astype(str)
avg_sent(avg_yr_df)
avg_yr_df['Volume'] = 1
avg_yr_df = avg_yr_df.groupby('Year').agg({'Avg_Sentiment': 'mean', 'Volume': 'sum'}).reset_index()
# only data from 2019 onwards
avg_yr_df = avg_yr_df[avg_yr_df['Year'] >= '2019'].reset_index(drop=True)
# round to 2 decimal places
avg_yr_df['Avg_Sentiment'] = avg_yr_df['Avg_Sentiment'].round(2)
avg_yr_df.T

/var/folders/4c/w2fyysz50_j_234jdvb_c9wr0000gn/T/ipykernel_37713/3966298676.py:2: DtypeWarning:

Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.

/var/folders/4c/w2fyysz50_j_234jdvb_c9wr0000gn/T/ipykernel_37713/3966298676.py:3: UserWarning:

Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.



,0,1,2,3,4
Year,2019,2020,2021,2022,2023
Avg_Sentiment,0.05,0.04,-0.04,0.14,-0.05
Volume,5360,11134,8273,4719,34820
